In [ ]:
# Reorder the centroids
def reorder_points_by_columns(input_geojson, output_geojson, rotation_angle=0):
    """
    Reorder points in a GeoJSON file such that points are ordered column-first.
    The rotation is used only for ordering purposes; the final output retains original coordinates.
    
    Parameters:
        input_geojson (str): Path to the input GeoJSON file containing points.
        output_geojson (str): Path to the output GeoJSON file with reordered points.
        rotation_angle (float): Angle in degrees to rotate the grid for ordering (default: 0).
    
    Returns:
        None
    """
    # Load GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(input_geojson)
    
    # Compute the centroid of the whole grid
    grid_centroid = MultiPoint(gdf.geometry).centroid
    
    # Rotate the points temporarily for ordering
    if rotation_angle != 0:
        gdf['rotated_geometry'] = gdf['geometry'].apply(
            lambda geom: rotate(geom, rotation_angle, origin=grid_centroid, use_radians=False)
        )
    else:
        gdf['rotated_geometry'] = gdf['geometry']
    
    # Extract coordinates from rotated geometries for sorting
    gdf['x'] = gdf['rotated_geometry'].apply(lambda geom: geom.x)
    gdf['y'] = gdf['rotated_geometry'].apply(lambda geom: geom.y)
    
    # Sort by x (columns) first, and then by y (rows) within each column
    gdf_sorted = gdf.sort_values(by=['x', 'y'], ascending=[True, False]).reset_index(drop=True)
    
    # Drop temporary columns for sorting
    gdf_sorted = gdf_sorted.drop(columns=['rotated_geometry', 'x', 'y'])
    
    # Save the reordered GeoDataFrame as a new GeoJSON file
    gdf_sorted.to_file(output_geojson, driver="GeoJSON")

In [ ]:
def reorder_points_by_columns(input_geojson, output_geojson, rotation_angle=0, id_start=100, step=1):
    """
    Reorder points in a GeoJSON file such that points are ordered column-first.
    The rotation is used only for ordering purposes; the final output retains original coordinates.
    
    Parameters:
        input_geojson (str): Path to the input GeoJSON file containing points.
        output_geojson (str): Path to the output GeoJSON file with reordered points.
        rotation_angle (float): Angle in degrees to rotate the grid for ordering (default: 0).
        id_start (int): Starting value for IDs (default: 100).
        step (int): Step value to increment the ID for each column (default: 1).
    
    Returns:
        None
    """
    # Load GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(input_geojson)
    
    # Compute the centroid of the whole grid
    grid_centroid = MultiPoint(gdf.geometry).centroid
    
    # Rotate the points temporarily for sorting (order purposes only)
    if rotation_angle != 0:
        gdf['rotated_geometry'] = gdf['geometry'].apply(
            lambda geom: rotate(geom, rotation_angle, origin=grid_centroid, use_radians=False)
        )
    else:
        gdf['rotated_geometry'] = gdf['geometry']
    
    # Extract coordinates from rotated geometries for sorting
    gdf['x'] = gdf['rotated_geometry'].apply(lambda geom: geom.x)
    gdf['y'] = gdf['rotated_geometry'].apply(lambda geom: geom.y)
    
    # Sort by x (columns) first, and then by y (rows) within each column
    gdf_sorted = gdf.sort_values(by=['x', 'y'], ascending=[True, False]).reset_index(drop=True)
    
    # Drop temporary columns for sorting
    gdf_sorted = gdf_sorted.drop(columns=['rotated_geometry', 'x', 'y'])
    
    # Add an ID column based on column-first order
    id_list = []
    current_id = id_start
    
    last_x = None  # Track the last x-coordinate to detect new columns
    
    for index, row in gdf_sorted.iterrows():
        # If the x-coordinate changes (indicating a new column), increment the ID
        if last_x is None or row.geometry.x != last_x:
            current_id = id_start
        
        id_list.append(current_id)
        current_id += step
        
        last_x = row.geometry.x
    
    gdf_sorted['ID'] = id_list
    
    # Save the reordered GeoDataFrame with updated IDs as a new GeoJSON file
    gdf_sorted.to_file(output_geojson, driver="GeoJSON")